<a href="https://colab.research.google.com/github/SophistryDude/DS-Unit-2-Linear-Models/blob/master/module3-ridge-regression/Nick_M's_Copy_of_LS_DS_213_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 1, Module 3*

---

# Ridge Regression

## Assignment

We're going back to our other **New York City** real estate dataset. Instead of predicting apartment rents, you'll predict property sales prices.

But not just for condos in Tribeca...

- [ ] Use a subset of the data where `BUILDING_CLASS_CATEGORY` == `'01 ONE FAMILY DWELLINGS'` and the sale price was more than 100 thousand and less than 2 million.
- [ ] Do train/test split. Use data from January — March 2019 to train. Use data from April 2019 to test.
- [ ] Do one-hot encoding of categorical features.
- [ ] Do feature selection with `SelectKBest`.
- [ ] Fit a ridge regression model with multiple features. Use the `normalize=True` parameter (or do [feature scaling](https://scikit-learn.org/stable/modules/preprocessing.html) beforehand — use the scaler's `fit_transform` method with the train set, and the scaler's `transform` method with the test set)
- [ ] Get mean absolute error for the test set.
- [ ] As always, commit your notebook to your fork of the GitHub repo.

The [NYC Department of Finance](https://www1.nyc.gov/site/finance/taxes/property-rolling-sales-data.page) has a glossary of property sales terms and NYC Building Class Code Descriptions. The data comes from the [NYC OpenData](https://data.cityofnewyork.us/browse?q=NYC%20calendar%20sales) portal.


## Stretch Goals

Don't worry, you aren't expected to do all these stretch goals! These are just ideas to consider and choose from.

- [ ] Add your own stretch goal(s) !
- [ ] Instead of `Ridge`, try `LinearRegression`. Depending on how many features you select, your errors will probably blow up! 💥
- [ ] Instead of `Ridge`, try [`RidgeCV`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html).
- [ ] Learn more about feature selection:
    - ["Permutation importance"](https://www.kaggle.com/dansbecker/permutation-importance)
    - [scikit-learn's User Guide for Feature Selection](https://scikit-learn.org/stable/modules/feature_selection.html)
    - [mlxtend](http://rasbt.github.io/mlxtend/) library
    - scikit-learn-contrib libraries: [boruta_py](https://github.com/scikit-learn-contrib/boruta_py) & [stability-selection](https://github.com/scikit-learn-contrib/stability-selection)
    - [_Feature Engineering and Selection_](http://www.feat.engineering/) by Kuhn & Johnson.
- [ ] Try [statsmodels](https://www.statsmodels.org/stable/index.html) if you’re interested in more inferential statistical approach to linear regression and feature selection, looking at p values and 95% confidence intervals for the coefficients.
- [ ] Read [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf), Chapters 1-3, for more math & theory, but in an accessible, readable way.
- [ ] Try [scikit-learn pipelines](https://scikit-learn.org/stable/modules/compose.html).

In [2]:
%%capture

import sys
import pandas as pd
import pandas_profiling
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.feature_selection import SelectKBest
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
from category_encoders import OneHotEncoder
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [3]:
# Read New York City property sales data
df = pd.read_csv(DATA_PATH+'condos/NYC_Citywide_Rolling_Calendar_Sales.csv',
                 parse_dates=['SALE DATE'],  
                 index_col='SALE DATE')

# Change column names: replace spaces with underscores
df.columns = [col.replace(' ', '_') for col in df]

# SALE_PRICE was read as strings.
# Remove symbols, convert to integer
df['SALE_PRICE'] = (
    df['SALE_PRICE']
    .str.replace('$','')
    .str.replace('-','')
    .str.replace(',','')
    .astype(int)
)

In [4]:
# BOROUGH is a numeric column, but arguably should be a categorical feature,
# so convert it from a number to a string
df['BOROUGH'] = df['BOROUGH'].astype(str)

In [5]:
# Reduce cardinality for NEIGHBORHOOD feature

# Get a list of the top 10 neighborhoods
top10 = df['NEIGHBORHOOD'].value_counts()[:10].index

# At locations where the neighborhood is NOT in the top 10, 
# replace the neighborhood with 'OTHER'
df.loc[~df['NEIGHBORHOOD'].isin(top10), 'NEIGHBORHOOD'] = 'OTHER'

We need to know how the database handles the sale price numbers so we will sort in descending rate and take a look at the chart. 

In [6]:
df.info()
df.sort_values(by='SALE_PRICE', ascending=False).head()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 23040 entries, 2019-01-01 to 2019-04-30
Data columns (total 20 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   BOROUGH                         23040 non-null  object 
 1   NEIGHBORHOOD                    23040 non-null  object 
 2   BUILDING_CLASS_CATEGORY         23040 non-null  object 
 3   TAX_CLASS_AT_PRESENT            23039 non-null  object 
 4   BLOCK                           23040 non-null  int64  
 5   LOT                             23040 non-null  int64  
 6   EASE-MENT                       0 non-null      float64
 7   BUILDING_CLASS_AT_PRESENT       23039 non-null  object 
 8   ADDRESS                         23040 non-null  object 
 9   APARTMENT_NUMBER                5201 non-null   object 
 10  ZIP_CODE                        23039 non-null  float64
 11  RESIDENTIAL_UNITS               23039 non-null  float64
 12  COMMERCIAL_UNIT

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE
SALE DATE,,,,,,,,,,,,,,,,,,,,
2019-02-08,1,OTHER,22 STORE BUILDINGS,4,840,42,NaN,K3,424 5 AVENUE,NaN,10018.0,0.0,4.0,4.0,"51,271",697029.0,1920.0,4,K3,850000000
2019-01-08,1,OTHER,21 OFFICE BUILDINGS,4,1321,22,NaN,O4,885 2 AVENUE,NaN,10017.0,0.0,27.0,27.0,"40,820",805467.0,1971.0,4,O4,565754371
2019-01-08,1,OTHER,21 OFFICE BUILDINGS,4,1306,33,NaN,O4,850 THIRD AVENUE,NaN,10022.0,0.0,21.0,21.0,"31,632",574675.0,1960.0,4,O4,422000000
2019-01-23,1,OTHER,25 LUXURY HOTELS,4,1009,19,NaN,H1,118 WEST 57TH STREET,NaN,10019.0,0.0,9.0,9.0,"22,092",536719.0,1939.0,4,H1,389858000
2019-03-11,4,OTHER,08 RENTALS - ELEVATOR APARTMENTS,2,422,7,NaN,D6,42-12 28TH STREET,NaN,11101.0,450.0,1.0,451.0,"17,500",483148.0,2015.0,2,D6,335000000


It makes sense to me to isolate our building category to single family dwellings when setting up our new dataframe. Using this method we're going to change our focus a little bit, since one of the key components of what we're looking at is already isolated. 

In [7]:
sfd = df[(df['SALE_PRICE'] >= 100000) & (df['SALE_PRICE'] <= 2000000) & 
         (df['BUILDING_CLASS_CATEGORY'] == '01 ONE FAMILY DWELLINGS')];

In [8]:
print(sfd.shape)
sfd.info()

(3164, 20)
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3164 entries, 2019-01-01 to 2019-04-30
Data columns (total 20 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   BOROUGH                         3164 non-null   object 
 1   NEIGHBORHOOD                    3164 non-null   object 
 2   BUILDING_CLASS_CATEGORY         3164 non-null   object 
 3   TAX_CLASS_AT_PRESENT            3164 non-null   object 
 4   BLOCK                           3164 non-null   int64  
 5   LOT                             3164 non-null   int64  
 6   EASE-MENT                       0 non-null      float64
 7   BUILDING_CLASS_AT_PRESENT       3164 non-null   object 
 8   ADDRESS                         3164 non-null   object 
 9   APARTMENT_NUMBER                1 non-null      object 
 10  ZIP_CODE                        3164 non-null   float64
 11  RESIDENTIAL_UNITS               3164 non-null   float64
 12  COMME

Here we have narrowed down our 23,040 entries to just 3,164. Using this new dataset we can analyze other factors. Since Sq Ft is the industry standard, I'm going to use that as one of my x variables. 

In [9]:
target = 'SALE_PRICE'
y = sfd[target]
x = sfd.drop([target]+['TAX_CLASS_AT_PRESENT', 'EASE-MENT', 'TAX_CLASS_AT_TIME_OF_SALE', 'BUILDING_CLASS_AT_TIME_OF_SALE', 'APARTMENT_NUMBER', 'BUILDING_CLASS_AT_PRESENT', 'ADDRESS', 'YEAR_BUILT', 'RESIDENTIAL_UNITS', 'COMMERCIAL_UNITS', 'TOTAL_UNITS', 'LAND_SQUARE_FEET'], axis=1)
cutoff = '2019-03-31'
mask = x.index < cutoff
x_train = x.loc[mask]
y_train = y.loc[mask]
x_val, y_val = x.loc[~mask], y.loc[~mask]

col = 'GROSS_SQUARE_FEET'
y_pred = [y_train.mean()]*len(x_train)
print('Baseline MAE:', mean_absolute_error(y_train, y_pred))
model = LinearRegression()
model.fit(x_train[[col]], y_train)
print('Training MAE:', mean_absolute_error(y_train, model.predict(x_train[[col]])))
print('Validation MAE:', mean_absolute_error(y_val, model.predict(x_val[[col]])))
print('Training RMSE:', mean_squared_error(y_train, model.predict(x_train[[col]]), squared=False))
print('Validation RMSE:', mean_squared_error(y_val, model.predict(x_val[[col]]), squared=False))
print('Training R2:', model.score(x_train[[col]], y_train))
print('Validation R2:', model.score(x_val[[col]], y_val))


Baseline MAE: 216497.12357684076
Training MAE: 195029.1438991597
Validation MAE: 185788.22229822693
Training RMSE: 262958.15771295485
Validation RMSE: 268070.62504900654
Training R2: 0.2055472083645218
Validation R2: 0.21375946518825703


# OneHot Encoding

In [10]:
x_train.columns

Index(['BOROUGH', 'NEIGHBORHOOD', 'BUILDING_CLASS_CATEGORY', 'BLOCK', 'LOT',
       'ZIP_CODE', 'GROSS_SQUARE_FEET'],
      dtype='object')

In [11]:
transformer = OneHotEncoder(use_cat_names=True, cols=['NEIGHBORHOOD', 'BOROUGH', 'BLOCK', 'BUILDING_CLASS_CATEGORY'])

transformer.fit(x_train)
xt_train = transformer.transform(x_train)


In [12]:
xt_train.columns

Index(['BOROUGH_3', 'BOROUGH_4', 'BOROUGH_2', 'BOROUGH_5', 'BOROUGH_1',
       'NEIGHBORHOOD_OTHER', 'NEIGHBORHOOD_FLUSHING-NORTH',
       'NEIGHBORHOOD_EAST NEW YORK', 'NEIGHBORHOOD_BEDFORD STUYVESANT',
       'NEIGHBORHOOD_FOREST HILLS',
       ...
       'BLOCK_6220.0', 'BLOCK_5720.0', 'BLOCK_5449.0', 'BLOCK_54.0',
       'BLOCK_2373.0', 'BLOCK_3395.0', 'BLOCK_11159.0', 'LOT', 'ZIP_CODE',
       'GROSS_SQUARE_FEET'],
      dtype='object', length=2082)

In [13]:
transformkbest = SelectKBest(k=10)
transformkbest.fit(xt_train, y_train)
xtt_train = transformkbest.transform(xt_train)

/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:114: UserWarning: Features [   4   12   19   25   34   43   55   60   66   67   68   71   74   75
   87   94   95   97  108  114  134  145  156  158  160  165  178  186
  188  195  204  211  213  231  234  236  242  244  247  259  263  276
  291  300  303  311  319  322  325  335  338  346  347  354  356  358
  359  367  375  378  381  390  393  397  412  419  434  435  440  446
  448  454  463  469  472  479  483  484  487  488  489  499  507  510
  512  515  536  538  542  549  556  563  565  566  572  577  579  580
  584  585  587  590  592  594  599  600  607  608  614  623  627  636
  639  646  649  651  660  663  666  667  674  685  688  708  712  713
  714  718  750  759  773  784  785  791  793  799  804  806  809  817
  818  827  828  829  831  837  839  842  844  853  854  857  861  865
  870  874  881  893  894  897  916  919  928  929  931  934  943  948
  949  950  951  952  956  969 

In [14]:
model.fit(xtt_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

# Ridge Regression

In [17]:
predictor = Ridge(alpha=0.1)

predictor.fit(xtt_train, y_train)


Ridge(alpha=0.1, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=None, solver='auto', tol=0.001)

In [19]:
y_pred = predictor.predict(xtt_train)
print('Training MAE:', mean_absolute_error(y_train, y_pred))


Training MAE: 215081.38835855745


In [22]:
xt_val = transformer.tranform(x_val)
xtt_val = transformkbest.transform(xt_val)
y_pred = predictor.predict(xtt_val)
print('Training MAE:', mean_absolute_error(y_val, y_pred))

NameError: ignored